In [1]:
import pandas as pd
import io
import requests
import numpy as np

In [2]:
url="https://raw.githubusercontent.com/JULIELab/EmoBank/master/corpus/emobank.csv"
s=requests.get(url).content
emodata=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
emodata.head()

,id,V,A,D,text
0,110CYL068_1036_1079,3.00,3.00,3.20,"Remember what she said in my last letter? """
1,110CYL068_1079_1110,2.80,3.10,2.80,If I wasn't working here.
2,110CYL068_1127_1130,3.00,3.00,3.00,".."""
3,110CYL068_1137_1188,3.44,3.00,3.22,Goodwill helps people get off of public assist...
4,110CYL068_1189_1328,3.55,3.27,3.46,Sherry learned through our Future Works class ...


The columns V, A and D represent Valence (negative vs. positive), Arousal (calm vs. excited), and Dominance (being controlled vs. being in control). Each of those take numeric values from [1, 5]. Please refer to the paper for further details.

In [4]:
emodata = emodata[['V', 'A', 'text']]
max_v = max(emodata.V)
max_a = max(emodata.A)
emodata.V = emodata.V / max_v
emodata.A = emodata.A / max_a
#emodata = emodata.rename(columns={'V': 'Neutral', 'A': 'Position'})
emodata.head()

,V,A,text
0,0.652174,0.681818,"Remember what she said in my last letter? """
1,0.608696,0.704545,If I wasn't working here.
2,0.652174,0.681818,".."""
3,0.747826,0.681818,Goodwill helps people get off of public assist...
4,0.771739,0.743182,Sherry learned through our Future Works class ...


In [5]:
sentiments = pd.read_csv('Sentiment_Data.csv')
sentiments.head()

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons


In [6]:
max_Negative = max(sentiments.Negative)
max_Neutral = max(sentiments.Neutral)
max_Positive = max(sentiments.Positive)
max_val = max(max_Negative, max_Neutral, max_Positive)
sentiments.Negative = sentiments.Negative / max_val
sentiments.Neutral = sentiments.Neutral / max_val
sentiments.Positive = sentiments.Positive / max_val

In [7]:
sentiments.to_csv("text.csv")

In [8]:
a = 0.33
b = 0.66
emodata['Emoji'] = 'None'
emodata['Positive'] = 'None'
emodata['Neutral'] = 'None'
emodata['Negative'] = 'None'
for i in range(len(emodata)):
    index = 0
    correctness = 1
    if emodata.loc[i, 'V'] <= a:
        neg_to_neut = emodata.loc[i, 'V'] / a
        for j in range(51):
            if sentiments.loc[j, 'Negative'] > sentiments.loc[j, 'Positive'] and sentiments.loc[j, 'Negative'] > sentiments.loc[j, 'Neutral'] :
                neg_to_neut_j = sentiments.loc[j, 'Neutral'] / sentiments.loc[j, 'Negative']
                if abs(neg_to_neut - neg_to_neut_j) < correctness:
                    correctness = abs(neg_to_neut - neg_to_neut_j)
                    index = j
    if a < emodata.loc[i, 'V'] <= b:
        neg_to_neut = a / emodata.loc[i, 'V']
        pos_to_neut = emodata.loc[i, 'V'] / b
        for j in range(51):
            if sentiments.loc[j, 'Neutral'] > sentiments.loc[j, 'Negative'] and sentiments.loc[j, 'Neutral'] > sentiments.loc[j, 'Positive'] :
                neg_to_neut_j = sentiments.loc[j, 'Negative'] / sentiments.loc[j, 'Neutral']
                pos_to_neut_j = sentiments.loc[j, 'Positive'] / sentiments.loc[j, 'Neutral']
                if abs(neg_to_neut - neg_to_neut_j) < correctness and abs(pos_to_neut - pos_to_neut_j) < correctness:
                    correctness = max(abs(neg_to_neut - neg_to_neut_j), abs(pos_to_neut - pos_to_neut_j))
                    index = j
    if emodata.loc[i, 'V'] > b:
        pos_to_neut = b / emodata.loc[i, 'V']
        for j in range(51):
            if sentiments.loc[j, 'Positive'] > sentiments.loc[j, 'Neutral'] and sentiments.loc[j, 'Positive'] > sentiments.loc[j, 'Negative'] :
                pos_to_neut_j = sentiments.loc[j, 'Neutral'] / sentiments.loc[j, 'Positive']
                if abs(pos_to_neut - pos_to_neut_j) < correctness:
                    correctness = abs(pos_to_neut - pos_to_neut_j)
                    index = j
            
    emodata.loc[i, 'Emoji'] = sentiments.loc[index, 'Emoji']
    emodata.loc[i, 'Positive'] = sentiments.loc[index, 'Positive']
    emodata.loc[i, 'Neutral'] = sentiments.loc[index, 'Neutral']
    emodata.loc[i, 'Negative'] = sentiments.loc[index, 'Negative']

In [9]:
emodata

,V,A,text,Emoji,Positive,Neutral,Negative
0,0.652174,0.681818,"Remember what she said in my last letter? """,✨,0.0499635,0.0676406,0.00628196
1,0.608696,0.704545,If I wasn't working here.,✨,0.0499635,0.0676406,0.00628196
2,0.652174,0.681818,"..""",✨,0.0499635,0.0676406,0.00628196
3,0.747826,0.681818,Goodwill helps people get off of public assist...,🙏,0.112053,0.0946676,0.0181154
4,0.771739,0.743182,Sherry learned through our Future Works class ...,🙏,0.112053,0.0946676,0.0181154
5,0.782609,0.750000,Coming to Goodwill was the first step toward m...,🙏,0.112053,0.0946676,0.0181154
6,0.652174,0.681818,"I am now... totally off of welfare.""",✨,0.0499635,0.0676406,0.00628196
7,0.673913,0.704545,Goodwill prepares people for life-long employm...,☀,0.0566837,0.0550767,0.00306793
8,0.706522,0.654545,Here's another story of success from what migh...,😳,0.0426589,0.0404675,0.0404675
9,0.706522,0.681818,Cornell found a number of employment options t...,😳,0.0426589,0.0404675,0.0404675


In [10]:
emodata.to_csv('emodata_050519.csv')

## later is old code

In [6]:
max(sentiments.Neutral)

4163

In [7]:
sentiments = sentiments[['Emoji', 'Position', 'Neutral', 'Unicode name']]
sentiments.head()

,Emoji,Position,Neutral,Unicode name
0,😂,0.805101,4163,FACE WITH TEARS OF JOY
1,❤,0.746943,1334,HEAVY BLACK HEART
2,♥,0.753806,1942,BLACK HEART SUIT
3,😍,0.765292,1390,SMILING FACE WITH HEART-SHAPED EYES
4,😭,0.803352,1218,LOUDLY CRYING FACE


In [8]:
# norm Neutral col
max_val = max(sentiments.Neutral)
sentiments.Neutral = sentiments.Neutral / max_val
sentiments.head()

,Emoji,Position,Neutral,Unicode name
0,😂,0.805101,1.000000,FACE WITH TEARS OF JOY
1,❤,0.746943,0.320442,HEAVY BLACK HEART
2,♥,0.753806,0.466491,BLACK HEART SUIT
3,😍,0.765292,0.333894,SMILING FACE WITH HEART-SHAPED EYES
4,😭,0.803352,0.292577,LOUDLY CRYING FACE


In [9]:
def mergeDfs(left, right): 
    left['Emoji'] = 0
    for i in range(len(left)):
        min_dist = 100000
        for j in range(len(right)):
            diff_Position = left.Position[i] - right.Position[j]
            diff_Neutral = left.Neutral[i] - right.Neutral[j]
            euclid_dist = np.power((np.power(diff_Position, 2) + np.power(diff_Neutral, 2)), 0.5)
            if euclid_dist < min_dist:
                min_dist = euclid_dist
                index = j
        left.loc[i, 'Emoji'] = right.loc[index, 'Emoji']  
    return left

In [10]:
df = mergeDfs(emodata, sentiments)

In [11]:
df.head()

,Neutral,Position,text,Emoji
0,0.652174,0.681818,"Remember what she said in my last letter? """,♥
1,0.608696,0.704545,If I wasn't working here.,♥
2,0.652174,0.681818,"..""",♥
3,0.747826,0.681818,Goodwill helps people get off of public assist...,😂
4,0.771739,0.743182,Sherry learned through our Future Works class ...,😂


In [12]:
df.to_csv('merged.csv')

In [13]:
df.Emoji.unique()

array(['♥', '😂', '😍', '😘', '😭', '❤'], dtype=object)

In [14]:
sentiments.Emoji.unique()

array(['😂', '❤', '♥', '😍', '😭', '😘', '😊', '👌', '💕', '👏', '😁', '☺', '♡',
       '👍', '😩', '🙏', '✌', '😏', '😉', '🙌', '🙈', '💪', '😄', '😒', '💃', '💖',
       '😃', '😔', '😱', '🎉', '😜', '☯', '🌸', '💜', '💙', '✨', '😳', '💗', '★',
       '█', '☀', '😡', '😎', '😢', '💋', '😋', '🙊', '😴', '🎶', '💞', '😌', '🔥',
       '💯', '🔫', '💛', '💁', '💚', '♫', '😞', '😆', '😝', '😪', '�', '😫', '😅',
       '👊', '💀', '😀', '😚', '😻', '©', '👀', '💘', '🐓', '☕', '👋', '✋', '🎊',
       '🍕', '❄', '😥', '😕', '💥', '💔', '😤', '😈', '►', '✈', '🔝', '😰', '⚽',
       '😑', '👑', '😹', '👉', '🍃', '🎁', '😠', '🐧', '☆', '🍀', '🎈', '🎅', '😓',
       '😣', '😐', '✊', '😨', '😖', '💤', '💓', '👎', '💦', '✔', '😷', '⚡', '🙋',
       '🎄', '💩', '🎵', '➡', '😛', '😬', '👯', '💎', '🌿', '🎂', '🌟', '🔮', '❗',
       '👫', '🏆', '✖', '☝', '😙', '⛄', '👅', '♪', '🍂', '💏', '🔪', '🌴', '👈',
       '🌹', '🙆', '➜', '👻', '💰', '🍻', '🙅', '🌞', '🍁', '⭐', '▪', '🎀', '━',
       '☷', '🐷', '🙉', '🌺', '💅', '🐶', '🌚', '👽', '🎤', '👭', '🎧', '👆', '🍸',
       '🍷', '®', '🍉', '😇', '☑', '🏃', '😿', '│', '💣', '🍺', '▶', '😲